In [1]:
import os
import sys
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
sys.path.append("../")
# from tools.name_data import NameDataset

/Users/bruce/anaconda3/envs/pytorch/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 隐藏节点100， 2层，batch_size=256，100个epoch
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_EPOCHS = 100

# 数据处理

In [3]:
class NameDataset:
    def __init__(self, is_train=False):
        filename = "/Users/bruce/PycharmProjects/Pytorch_learning/data/names_train.csv.gz" if is_train else "/Users/bruce/PycharmProjects/Pytorch_learning/data/names_test.csv.gz"
        with gzip.open(filename, 'rt') as f:
            pass

In [17]:
import csv
import gzip
filename = "/Users/bruce/PycharmProjects/Pytorch_learning/data/names_train.csv.gz"
with gzip.open(filename, 'rt') as f:
    reader = csv.reader(f)
    rows = list(reader)

    
names = [row[0] for row in rows]
labels = [row[1] for row in rows]
total_len = len(labels)
print('name is: ', len(names), names[0:10])
print('labels is: ', len(labels), labels[0:10])

name is:  13374 ['Adsit', 'Ajdrna', 'Antonowitsch', 'Antonowitz', 'Ballalatak', 'Ballaltick', 'Bastl', 'Baroch', 'Betlach', 'Biganska']
labels is:  13374 ['Czech', 'Czech', 'Czech', 'Czech', 'Czech', 'Czech', 'Czech', 'Czech', 'Czech', 'Czech']


In [20]:
countries = list(sorted(list(set(labels))))
countries.__len__()

18

In [260]:
countries

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese']

In [261]:
def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)

## 定义一个数据类

In [280]:
class NameDataset1:
    def __init__(self, is_train=False):
        filename = "../../data/names_train.csv.gz" if is_train else "../../data/names_test.csv.gz" 
        with gzip.open(filename, 'rt') as f:
            reader = csv.reader(f)
            self.rows = list(reader)
        self.names = [row[0] for row in self.rows]
        self.labels = [row[1] for row in self.rows]
        self.country = list(sorted(set(labels)))
        self.total_len = len(self.names)

    def __getitem__(self, index):
        return str2ascii_arr(self.names[index]), self.labels[index]
    
    def __len__(self):
        return self.total_len
    
    def get_countries(self):
        return self.country
    
    def get_country(slef, idx):
        return self.country[idx]
    
    def get_county_id(self, name):
        return self.country.index(name)

In [281]:
train_data = NameDataset1(True)
test_data = NameDataset1()

In [282]:
train_data.get_countries()

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese']

In [293]:
test_data = NameDataset1()

In [294]:
train_data[18]

(([66, 111, 114, 111, 118, 115, 107, 105], 8), 'Czech')

In [295]:
train_data[18][0][1]

8

In [296]:
train_data[18][1]

'Czech'

In [ ]:
def collate_fn(train_data):
    """
    该函数的功能就是对train_data数据进行填充，填充原则是对当前批次的数据长度先要进行排序
    按照从大到小的顺序排序，然后开始填充
    :param train_data:
    :return:
    """

    # 按照长度排序
    train_data.sort(key=lambda data: len(data), reverse=True)

    # 得到排序后的数据的长度列表
    data_length = [len(data) for data in train_data]

    # 对该数据进行填充
    train_data = torch.nn.utils.rnn.pad_sequence(train_data, batch_first=True, padding_value=0)

    print('train data shape: ', train_data.shape)
    # 对train_data最后增加了一维数据，返回[batch_size, seq_len, input_size]数据和长度
    return train_data.unsqueeze(-1), data_length


In [284]:
train_dataloader = DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=3, shuffle=True)

for (name, label) in (train_dataloader):
    print(seq_len)
    print(name)
    print(label)

RuntimeError: each element in list of batch should be of equal size

In [246]:
def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)

In [258]:
import numpy as np
for i, (names, label) in enumerate(train_dataloader):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    inputdata = [torch.Tensor(sl[0]) for sl in sequence_and_length]
    seq_len = np.array([sl[1] for sl in sequence_and_length])
    print('input ',inputdata)
    print('seq len ', seq_len)

input  [tensor([ 68., 105., 107.]), tensor([ 90., 104., 101., 122., 104., 101., 108.]), tensor([ 74., 117., 107.,  97., 108., 111., 118.])]
seq len  [3 7 7]
input  [tensor([ 76., 101., 119., 101., 114., 101., 110., 116., 122.]), tensor([ 79., 116., 116., 111.]), tensor([ 77., 105., 107., 104.,  97., 108., 121.,  99., 104., 101., 118.])]
seq len  [ 9  4 11]
input  [tensor([ 90., 104., 105., 116., 108., 117., 107., 104., 105., 110.]), tensor([ 89.,  97., 103., 111., 100.,  97.]), tensor([ 65., 113., 117., 105., 110., 111.])]
seq len  [10  6  6]
input  [tensor([ 65.,  98.,  98.,  97., 100., 101., 108., 108., 105.]), tensor([ 84., 105., 104., 111., 109., 105., 114., 110., 111., 118.]), tensor([ 65., 110., 117., 114., 105., 110.])]
seq len  [ 9 10  6]
input  [tensor([ 68., 101.,  32., 102., 105., 108., 105., 112., 112., 105., 115.]), tensor([ 82., 111., 115., 115., 105., 110., 105.]), tensor([ 82., 117., 115., 121.,  97., 101., 118.])]
seq len  [11  7  7]
input  [tensor([ 65.,  98., 121., 1

input  [tensor([ 80., 114., 105., 107.]), tensor([ 75.,  97., 116., 115., 117., 107.,  97., 119.,  97.]), tensor([ 79., 107.,  97., 122.,  97., 107., 105.])]
seq len  [4 9 7]
input  [tensor([ 71.,  97., 108., 108., 121.,  97., 109., 111., 118.]), tensor([ 82., 117.,  97., 100., 104.,  97., 110.]), tensor([ 72., 108., 111.,  98., 121., 115., 116., 105., 110.])]
seq len  [9 7 9]
input  [tensor([ 77., 105., 107., 104., 101., 101., 118.]), tensor([ 77., 111., 118.,  99., 104., 117., 110.]), tensor([ 84., 111., 107.,  97., 114., 115., 107., 121.])]
seq len  [7 7 8]
input  [tensor([ 80., 121., 106., 101., 118.]), tensor([ 83., 109., 105., 116., 104., 115., 111., 110.]), tensor([ 71., 114.,  97., 109., 109.,  97., 116.,  97., 107.,  97., 107., 105.,
        115.])]
seq len  [ 5  8 13]
input  [tensor([ 82., 106., 101., 115., 104., 111., 116.,  97., 114., 115., 107., 121.]), tensor([ 67., 108.,  97., 114., 107.]), tensor([ 72., 105., 114., 115.,  99., 104.])]
seq len  [12  5  6]
input  [tensor(

input  [tensor([ 65., 108., 102., 116.,  97., 110.]), tensor([ 83., 101., 114., 101., 100.,  97.]), tensor([ 83., 104., 101., 112., 112.,  97., 114., 100.])]
seq len  [6 6 8]
input  [tensor([ 82., 105., 110., 103.]), tensor([ 80., 114., 111., 118.,  97., 110.]), tensor([ 71., 104.,  97., 110., 101., 109.])]
seq len  [4 6 6]
input  [tensor([ 90., 104., 105., 100., 107., 105., 110.]), tensor([ 65., 117., 116., 108., 101., 118.]), tensor([ 83., 104., 111., 100.,  97.])]
seq len  [7 6 5]
input  [tensor([ 78.,  97., 116.,  97., 112., 111., 118.]), tensor([ 86., 101., 100., 101., 110., 111., 118.]), tensor([ 67., 111., 108., 108., 105., 101., 114.])]
seq len  [7 7 7]
input  [tensor([ 83., 104.,  97., 109., 111., 110.]), tensor([ 80., 105.,  99., 107.,  97., 114., 100.]), tensor([ 76.,  97., 119., 101., 114., 101., 110., 122.])]
seq len  [6 7 8]
input  [tensor([ 65., 108.,  97., 109.,  97., 110., 110., 105.]), tensor([ 83., 105., 109., 115.]), tensor([ 84., 117., 109.,  97.])]
seq len  [8 4 4

input  [tensor([ 67.,  97., 115., 115., 105., 100., 121.]), tensor([ 70.,  97., 114.,  98., 101., 114.]), tensor([ 65., 103.,  97., 112., 111., 110., 111., 102., 102.])]
seq len  [7 6 9]
input  [tensor([ 72., 105., 110., 100., 108., 101., 121.]), tensor([ 80., 105., 116., 117., 104., 105., 110.]), tensor([ 90., 104., 117., 112.,  97., 110., 101., 110., 107., 111.])]
seq len  [ 7  7 10]
input  [tensor([ 89., 117., 100.,  97., 107., 104., 105., 110.]), tensor([ 72.,  97., 106., 106.,  97., 114.]), tensor([ 71., 114., 105., 109., 115., 104.,  97., 119.])]
seq len  [8 6 8]
input  [tensor([ 73., 109., 117., 107., 111., 118.]), tensor([ 65., 118., 100., 101., 101., 102., 102.]), tensor([ 65.,  98., 100., 117., 108.,  97., 116., 105., 112., 111., 118.])]
seq len  [ 6  7 11]
input  [tensor([ 76., 101.,  98., 101., 114., 101., 104., 116.]), tensor([ 75.,  97., 117., 102., 109.,  97., 110.]), tensor([ 71., 105., 108., 108.,  97., 114., 100.])]
seq len  [8 7 7]
input  [tensor([ 66.,  97., 122.]),

input  [tensor([ 76., 117., 112., 101., 110., 107., 111.]), tensor([ 86., 105., 108., 108., 101.]), tensor([ 82., 121., 122., 104., 107., 111., 118.])]
seq len  [7 5 7]
input  [tensor([ 82., 111., 119., 108., 105., 110., 115., 111., 110.]), tensor([ 87.,  97., 119., 114., 122.,  97., 115., 122., 101., 107.]), tensor([ 72.,  97., 109.,  97., 103., 117.,  99., 104., 105.])]
seq len  [ 9 10  9]
input  [tensor([ 67., 111., 116., 116., 114., 101., 108., 108.]), tensor([ 77.,  97., 114., 116., 121., 110., 121., 117., 107.]), tensor([ 89.,  97.,  99., 104., 109., 101., 110., 101., 118.])]
seq len  [8 9 9]
input  [tensor([ 65., 108.,  98., 105., 116., 115., 107., 121.]), tensor([ 83., 108., 101., 105., 109.,  97., 110.]), tensor([ 72.,  97., 112., 107., 111., 118.])]
seq len  [8 7 6]
input  [tensor([ 75.,  97.,  98., 121., 115., 104.]), tensor([ 65., 110., 111., 112., 114., 105., 101., 110., 107., 111.]), tensor([ 66., 111., 116., 114., 111., 115.])]
seq len  [ 6 10  6]
input  [tensor([ 82., 1

...

input  [tensor([ 86., 114.,  97., 122., 101., 108.]), tensor([ 79., 119., 101., 110., 115.]), tensor([ 80., 101., 114.,  99., 105., 118.,  97., 108.])]
seq len  [6 5 8]
input  [tensor([ 77., 111., 114.,  99., 111., 115.]), tensor([ 65., 115., 102., 111., 117., 114.]), tensor([ 86., 101., 101., 118., 101., 114., 115.])]
seq len  [6 6 7]
input  [tensor([ 65., 114.,  97., 117., 106., 111.]), tensor([ 80., 114., 111., 107., 111., 102., 105., 101., 118.]), tensor([ 72., 111., 108., 122., 101., 114.])]
seq len  [6 9 6]
input  [tensor([ 68., 111., 114., 114., 105., 110., 103., 116., 111., 110.]), tensor([ 76., 101.,  99., 117., 121., 101., 114.]), tensor([ 78., 101., 115., 103., 111., 118., 111., 114., 111., 118.])]
seq len  [10  7 10]
input  [tensor([ 71.,  97., 108., 108.]), tensor([ 71., 105., 111., 108., 108.,  97.]), tensor([ 68., 111., 105., 110., 111.])]
seq len  [4 6 5]
input  [tensor([ 74.,  97., 115., 115., 111.]), tensor([ 90., 104., 105., 107., 104.,  97., 114., 101., 118., 105., 

input  [tensor([ 82., 101., 109.,  99., 104., 117., 107., 111., 118.]), tensor([ 77., 111., 108., 111., 116., 107., 111., 118.]), tensor([ 84., 117., 114., 118., 101., 121.])]
seq len  [9 8 6]
input  [tensor([ 89.,  97., 107., 105., 109., 111., 118.]), tensor([ 77.,  97., 114., 115., 104.,  97., 110., 115., 107., 121.]), tensor([ 90., 104., 117., 114., 107., 111., 118., 115., 107., 121.])]
seq len  [ 7 10 10]
input  [tensor([ 68., 105., 118., 101., 101., 118.]), tensor([ 75., 117., 112., 102., 101., 108.]), tensor([ 77., 117., 115., 104.,  97., 115., 104., 105.,  98., 111.])]
seq len  [ 6  6 10]
input  [tensor([ 72.,  97., 105., 108., 111., 118.]), tensor([ 79., 104.,  98.,  97.]), tensor([ 82., 101., 104.,  98., 105., 110., 100., 101., 114.])]
seq len  [6 4 9]
input  [tensor([ 65., 103.,  97., 110.,  98., 101., 103., 121.,  97., 110.]), tensor([ 75., 114., 101.,  98., 115.]), tensor([ 69., 108., 102., 109.,  97., 110.])]
seq len  [10  5  6]
input  [tensor([ 77., 105., 104.,  97., 105.

input  [tensor([ 67.,  97., 114., 114.]), tensor([ 77.,  99., 102.,  97., 100., 100., 101., 110.]), tensor([ 70., 117., 114., 108., 111., 110., 103.])]
seq len  [4 8 7]
input  [tensor([ 90., 104.,  97., 110., 103.]), tensor([ 90., 104.,  97.,  98., 111., 101., 118.]), tensor([ 89., 117., 107., 104., 118., 105., 100., 111., 118.])]
seq len  [5 7 9]
input  [tensor([ 66., 114.,  97., 100.,  97.,  99., 104.]), tensor([ 71., 105.,  97., 110., 110.,  97., 107., 111., 112., 111., 117., 108.,
        111., 115.]), tensor([ 83., 105., 110.,  99., 108.,  97., 105., 114.])]
seq len  [ 7 14  8]
input  [tensor([ 78.,  97., 115., 101., 116., 107., 105., 110.]), tensor([ 68., 111.,  98., 114., 121., 110., 105., 110.]), tensor([ 74.,  97., 107., 117., 115., 104., 105., 110.])]
seq len  [8 8 8]
input  [tensor([ 70., 111., 114., 115., 104., 116., 101., 116., 101., 114.]), tensor([ 65., 114., 116., 104., 117., 114.]), tensor([ 84., 111., 117., 109.,  97.])]
seq len  [10  6  5]
input  [tensor([ 72.,  97.,

input  [tensor([ 71., 105., 108.]), tensor([ 65., 100., 111., 114.,  97., 116., 115., 107., 105.]), tensor([ 87., 105.,  99., 107., 104.,  97., 109.])]
seq len  [3 9 7]
input  [tensor([ 77., 111., 114., 101., 108., 108., 111.]), tensor([ 83., 104.,  97., 109., 109.,  97., 115.]), tensor([ 77., 105., 110., 116., 115.])]
seq len  [7 7 5]
input  [tensor([ 89.,  97., 109.,  97., 104.,  97.]), tensor([ 82., 105., 116.,  99., 104., 105., 101.]), tensor([ 75.,  97., 108., 105., 110.,  97.])]
seq len  [6 7 6]
input  [tensor([ 80., 105., 115., 107., 108., 111., 118.]), tensor([ 67., 104., 117., 114., 115.,  97., 108., 111., 118.]), tensor([ 76., 105., 115., 104., 105., 110.])]
seq len  [7 9 6]
input  [tensor([ 76., 121., 115., 101., 110., 107., 111., 118.]), tensor([ 83., 104., 101., 110., 105., 110., 103.]), tensor([ 84., 115., 101., 105., 109., 101., 110.])]
seq len  [8 7 7]
input  [tensor([ 72., 101., 110., 100., 101., 114., 115., 111., 110.]), tensor([ 73., 118.,  97., 110., 111., 118.]), t

input  [tensor([ 70., 114., 101., 117., 110., 100.]), tensor([ 76., 105., 104., 111., 118.]), tensor([ 65.,  98.,  97., 110., 111.])]
seq len  [6 5 5]
input  [tensor([ 76., 108., 111., 121., 100.]), tensor([ 86.,  97., 110.,  97., 103.]), tensor([ 83.,  97., 105., 100., 117., 108., 108.,  97., 101., 118.])]
seq len  [ 5  5 10]
input  [tensor([ 74., 101., 119., 101., 108., 108.]), tensor([ 90., 111., 103.,  98., 121.]), tensor([ 65., 115., 119.,  97., 100.])]
seq len  [6 5 5]
input  [tensor([ 86., 101., 115., 105.,  99., 104.]), tensor([ 84., 117., 108., 110., 105., 107., 111., 118.]), tensor([ 74.,  97., 103., 117.,  98., 111., 118.])]
seq len  [6 8 7]
input  [tensor([ 74., 111., 108., 108., 101., 110.,  98., 101.,  99., 107.]), tensor([ 84., 117., 114.,  98.,  97., 110., 111., 118.]), tensor([ 65., 108., 108., 105., 110., 103., 116., 111., 110.])]
seq len  [10  8  9]
input  [tensor([ 90., 111., 108., 111., 116., 110., 105., 116., 122., 107., 121.]), tensor([ 67., 104., 105., 106., 101

In [115]:
print(len(train_dataloader.dataset))
print(len(test_dataloader.dataset))

13374
6700


## 定义网络结构

In [176]:
# 网络结构就用一层rnn来做即可；
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.n_direciton = int(bidirectional) + 1
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, seq_lengths):
        input = input.t()
        batch_size = input.size(1)
        
        hidden = self._init_hidden(self.hidden_size)
        
        embedded = self.embedding(input)
        
        gru_input = pack_padded_sequence(embedded, seq_lengths.cpu().numpy())
        
        self.gru.flatten_parameters()
        
        output, hidden = self.gru(gru_input, hidden)
        
        fc_output = self.fc(hidden[-1])
        
        return fc_output
    
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_direciton, batch_size, self.hidden_size)
        return create_variable(hidden)
    

def create_variable(tensor):
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)
    


In [177]:
from torchsummary import summary

model = RNNClassifier(5, 5, 2)

x = torch.randn([16, 5])
seq_length = torch.randint(0, 8, [16])
print(x.shape)
print(seq_length.shape)
print(seq_length)

torch.Size([16, 5])
torch.Size([16])
tensor([2, 0, 3, 5, 5, 1, 0, 3, 5, 6, 6, 0, 4, 2, 1, 1])


In [178]:
x

tensor([[-0.8508, -0.2561,  0.6982, -0.9821,  0.3165],
        [ 0.8549,  0.2632, -0.7808,  1.2554,  0.4633],
        [-0.3899,  0.0594, -1.9653, -0.0953,  0.0550],
        [-1.3337, -0.2647,  0.3572,  0.0220, -0.3077],
        [-1.5365, -0.5449,  0.6546,  1.8111,  0.2559],
        [ 0.3522,  0.3841,  0.4527, -0.5012, -0.2581],
        [ 2.1694, -0.9384, -0.1801,  0.1238,  0.6324],
        [-1.1627,  0.8702,  1.6542,  1.3264, -1.2541],
        [ 0.5737, -0.9979, -0.5395,  0.8879, -1.0057],
        [-0.4661, -1.4099, -2.3668,  1.5192, -2.0554],
        [ 0.8952, -1.8777, -0.0543,  0.5133, -1.0130],
        [-1.3890, -0.6708,  0.9892,  1.1598,  0.8731],
        [ 2.8606, -0.4759,  0.0344, -0.4780, -0.0954],
        [ 0.1754, -0.2299,  0.6783,  0.1103, -0.8852],
        [ 0.8678, -0.4040, -0.4320, -0.4290, -0.3452],
        [ 0.8323, -1.5695, -0.1892,  0.8648,  0.2798]])

In [179]:
model(x, seq_length)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)